In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

Using TensorFlow backend.


In [2]:
data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [3]:
wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
print(words[0:10])
n_words = len(words); 
n_words

['greenhouse', 'airplane', 'leg', 'europeanunion-russian', 'stampede', 'sherwan', 'instance', 'socially-conservative', 'briefly', 'operates']


35178

In [4]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

72


In [5]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [6]:
print(char2idx['k'])
print(char2idx['-'])

39
28


In [7]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

In [8]:
X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [9]:
print(X_char[0:10])

[[5, 48, 18, 18, 73, 13, 4, 16, 41, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [43, 29, 48, 49, 11, 43, 73, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [11, 18, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [18, 16, 48, 4, 49, 18, 43, 73, 16, 73, 29, 4, 73, 28, 48, 16, 41, 41, 29, 43, 73, 0, 0, 0, 0], [41, 19, 43, 54, 49, 18, 38, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [41, 13, 18, 48, 47, 43, 73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [29, 73, 41, 19, 43, 73, 59, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [41, 4, 59, 29, 43, 11, 11, 25, 28, 59, 4, 73, 41, 18, 48, 69, 43, 19, 29, 69, 18, 0, 0, 0, 0], [12, 48, 29, 18, 14, 11, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4, 49, 18, 48, 43, 19, 18, 41, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [10]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(74, embedded_char_vector_length, input_length=max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)

In [11]:
'''X1 = np.array([[1,0,0],[0,0,1],[0,1,0],[0,0,0],[0,0,0],[0,0,0]])
train_char_vectors = model.predict(X1)

print(train_char_vectors)'''

'X1 = np.array([[1,0,0],[0,0,1],[0,1,0],[0,0,0],[0,0,0],[0,0,0]])\ntrain_char_vectors = model.predict(X1)\n\nprint(train_char_vectors)'

In [12]:
X1 = np.array(X_char)
print(X1)
train_char_vectors = model.predict(X1)
        

[[ 5 48 18 ...  0  0  0]
 [43 29 48 ...  0  0  0]
 [11 18  5 ...  0  0  0]
 ...
 [12 11  4 ...  0  0  0]
 [11  4 47 ...  0  0  0]
 [59 48 18 ...  0  0  0]]


In [13]:
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

(35178, 10)


[[-0.0215998  -0.00438682 -0.00979589 ...  0.01112138  0.00081465
   0.02142598]
 [-0.0217279  -0.00438642 -0.0097164  ... -0.01236339 -0.02069264
   0.00707564]
 [-0.021729   -0.00440656 -0.0097755  ... -0.0075145  -0.00365245
  -0.02342905]
 ...
 [-0.02171018 -0.00439098 -0.00973171 ... -0.01083119 -0.00918074
   0.01261888]
 [-0.02171948 -0.00440942 -0.00978664 ... -0.02722013 -0.00492839
  -0.0201044 ]
 [-0.0218172  -0.00427514 -0.00926688 ...  0.00116493 -0.00901464
   0.03098662]]


In [14]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [15]:
def pred_new_vec() :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [16]:
a1 = pred_new_num_enc("name")
a2 = pred_new_num_enc("nam")

print(cosine_similarity(a1 , a2))

21
22
[[0.98432717]]


In [17]:
a3 = pred_new_num_enc("name")
a4 = pred_new_num_enc("nam-e")

print(cosine_similarity(a3 , a4))

21
20
[[0.98103376]]


In [18]:
a5 = pred_new_num_enc("address")
a6 = pred_new_num_enc("addr")

print(cosine_similarity(a5 , a6))

18
21
[[0.8101735]]


In [19]:
a7 = pred_new_num_enc("address")
a8 = pred_new_num_enc("adres")

print(cosine_similarity(a7 , a8))

18
20
[[0.74404905]]


In [21]:
a9 = pred_new_num_enc("contact")
a10 = pred_new_num_enc("phone-no")

print(cosine_similarity(a9 , a10))

18
17
[[0.57691874]]
